In [53]:
import numpy as np
import pandas as pd

In [54]:
moon = pd.read_csv('data/lunardata.txt',
                   converters={'month': '{:0>2}'.format, 'day': '{:0>2}'.format})
moon.head()


,year,month,day,luna
0,1992,01,04,0
1,1992,01,05,1
2,1992,01,06,2
3,1992,01,07,3
4,1992,01,08,4


In [55]:
seizures = pd.read_csv('data/mora.csv')
seizures.head()

,id,date,int
0,1,20211204,1
1,2,20211206,2
2,3,20211207,1
3,4,20211207,1
4,5,20211209,1


In [56]:
moon['date'] = (moon['year'].astype(str) + moon['month'].astype(str) + moon['day'].astype(str)).astype(int)
moon.head()

,year,month,day,luna,date
0,1992,01,04,0,19920104
1,1992,01,05,1,19920105
2,1992,01,06,2,19920106
3,1992,01,07,3,19920107
4,1992,01,08,4,19920108


In [57]:
conc = pd.merge(seizures, moon, on="date")
conc.head()

,id,date,int,year,month,day,luna
0,1,20211204,1,2021,12,04,1
1,2,20211206,2,2021,12,06,3
2,3,20211207,1,2021,12,07,4
3,4,20211207,1,2021,12,07,4
4,5,20211209,1,2021,12,09,6
